Method 1

In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 4.9 MB 16.3 MB/s 
     |████████████████████████████████| 1.3 MB 62.3 MB/s 
     |████████████████████████████████| 120 kB 69.1 MB/s 
     |████████████████████████████████| 6.6 MB 25.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=92b96a63fe136b7fb3f728dd5c47aa2f7500e3c3f7f5b731008aa3cfd86cf559
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
#Write some lines to encode (sentences 0 and 2 are both ideltical):
sen = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "He found a leprechaun in his walnut shell."
]
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
#Encoding:
sen_embeddings = model.encode(sen)
sen_embeddings.shape

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

(4, 768)

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
#let's calculate cosine similarity for sentence 0:
cosine_similarity(
    [sen_embeddings[0]],
    sen_embeddings[1:]
)

array([[0.33088908, 0.7219258 , 0.55483645]], dtype=float32)

Method 2

In [4]:
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

tf.compat.v1.disable_eager_execution()

def execute(tensor):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())

        return sess.run(tensor)


def embed(input):
    handle = "https://tfhub.dev/google/elmo/3"
    model = hub.load(handle)
    
    return execute(model.signatures["default"](input)["elmo"])


def word_to_sentence(embeddings):
    return embeddings.sum(axis=1)


def get_embeddings_elmo_nnlm(sentences):
    return word_to_sentence(embed(sentences))

def cos_sim(a, b):
    return np.inner(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))




In [5]:

if __name__ == "__main__":
    inputs = ["people read the book", "the book people read"]
    input = tf.convert_to_tensor(
    inputs
    )
    # Get embeddings corresponding to each sentences
    results = get_embeddings_elmo_nnlm(input)

In [6]:
cos_sim(results[0], results[1])

0.83875865